<a href="https://colab.research.google.com/github/LuisErnestoColchado/EnvironmentalPollution/blob/master/LSTME_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
df = pd.read_csv("/content/drive/My Drive/masterTesis/code/data/BeijingPM20100101_20151231.csv")
df = df[np.isfinite(df['PM_US Post'])]

In [0]:
data = df.iloc[:,[1,2,3,4,9,11,13,15]].values

In [6]:
print(data)

[[ 2.010e+03  1.000e+00  1.000e+00 ...  4.100e+01 -5.000e+00  8.900e-01]
 [ 2.010e+03  1.000e+00  2.000e+00 ...  3.800e+01 -4.000e+00  1.790e+00]
 [ 2.010e+03  1.000e+00  2.000e+00 ...  4.200e+01 -4.000e+00  2.680e+00]
 ...
 [ 2.015e+03  1.200e+01  3.100e+01 ...  7.300e+01 -6.000e+00  8.900e-01]
 [ 2.015e+03  1.200e+01  3.100e+01 ...  7.300e+01 -6.000e+00  1.780e+00]
 [ 2.015e+03  1.200e+01  3.100e+01 ...  7.900e+01 -6.000e+00  2.670e+00]]


In [7]:
print(data.shape)

(50387, 8)


In [0]:
#GET METEOROLOGICAL DATA 
meteorological = data[:,[6,7,5]]

In [9]:
print(meteorological)

[[-5.    0.89 41.  ]
 [-4.    1.79 38.  ]
 [-4.    2.68 42.  ]
 ...
 [-6.    0.89 73.  ]
 [-6.    1.78 73.  ]
 [-6.    2.67 79.  ]]


In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaledMeteorological = scaler.fit_transform(meteorological)

In [0]:
#GET CATEGORICAL DATA
categorical_month = data[:,1].astype(int)
categorical_hour = data[:,3].astype(int)

In [0]:
#ENCODE CATEGORICAL DATA TO ONE-HOT
months = 12
hours = 24
onehot_month = np.zeros((len(data), months))
onehot_hours = np.zeros((len(data),hours))
onehot_month[np.arange(len(data)),categorical_month-1] = 1.0
onehot_hours[np.arange(len(data)),categorical_hour] = 1.0

In [0]:
auxData = np.concatenate((scaledMeteorological,onehot_month,onehot_hours),axis=1) 

In [14]:
print(auxData)

[[2.29508197e-01 7.86796130e-04 3.97959184e-01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.45901639e-01 2.39615185e-03 3.67346939e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.45901639e-01 3.98762584e-03 4.08163265e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.13114754e-01 7.86796130e-04 7.24489796e-01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.13114754e-01 2.37827012e-03 7.24489796e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.13114754e-01 3.96974411e-03 7.85714286e-01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]]


In [0]:
#PM2.5
pm25 = data[:,4]
pm25 = np.nan_to_num(pm25)
pm25 = np.reshape(pm25,(len(data),1))

In [16]:
print(pm25)

[[129.]
 [148.]
 [159.]
 ...
 [203.]
 [212.]
 [235.]]


In [0]:
scaledPm25 = scaler.fit_transform(pm25)

In [18]:
print(scaledPm25)

[[0.12890232]
 [0.14803625]
 [0.1591138 ]
 ...
 [0.20342397]
 [0.21248741]
 [0.23564955]]


In [0]:
historicalData =  np.zeros((len(data),8))
historicalLabel = np.zeros((len(data),1))
row = 0
col = 0
for i in range (0,len(data)):
    j = 8
    col = 0
    while(i > 8 and j > 0):
        historicalData[row,col] = scaledPm25[i-j]
        col += 1
        j -= 1
    historicalLabel[row,0] = scaledPm25[i]
    row+= 1

In [20]:
print(historicalData)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.04229607 0.04733132 0.05740181 ... 0.11379658 0.13293051 0.16918429]
 [0.04733132 0.05740181 0.06847936 ... 0.13293051 0.16918429 0.20342397]
 [0.05740181 0.06847936 0.09063444 ... 0.16918429 0.20342397 0.21248741]]


In [0]:
fullData = np.concatenate((historicalData,auxData,historicalLabel),axis=1)

In [22]:
print(fullData)
fulldata = fullData[0:50000,:]
print(fullData)

[[0.         0.         0.         ... 0.         1.         0.12890232]
 [0.         0.         0.         ... 0.         0.         0.14803625]
 [0.         0.         0.         ... 0.         0.         0.1591138 ]
 ...
 [0.04229607 0.04733132 0.05740181 ... 0.         0.         0.20342397]
 [0.04733132 0.05740181 0.06847936 ... 1.         0.         0.21248741]
 [0.05740181 0.06847936 0.09063444 ... 0.         1.         0.23564955]]
[[0.         0.         0.         ... 0.         1.         0.12890232]
 [0.         0.         0.         ... 0.         0.         0.14803625]
 [0.         0.         0.         ... 0.         0.         0.1591138 ]
 ...
 [0.04229607 0.04733132 0.05740181 ... 0.         0.         0.20342397]
 [0.04733132 0.05740181 0.06847936 ... 1.         0.         0.21248741]
 [0.05740181 0.06847936 0.09063444 ... 0.         1.         0.23564955]]


In [0]:
np.savetxt("/content/drive/My Drive/masterTesis/code/data/data.csv", fullData, delimiter=",")

In [24]:
print(fulldata.shape)

(50000, 48)


In [25]:
print(fulldata[50,:])

[0.06042296 0.05236657 0.07049345 0.0715005  0.0755287  0.07250755
 0.07854985 0.05740181 0.16393443 0.03835631 0.55102041 1.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.02416918]


In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [27]:
!pip install keras==2.2.4

In [28]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, concatenate, Dropout
from keras.layers import LSTM
from keras.layers.core import Reshape

Using TensorFlow backend.


In [0]:
def lstme(xtrain,ytrain,auxdata):
  inputs = Input((1,xtrain.shape[1]))

  
  #~inputs = Embedding(output_dim=xtrain.shape[0], input_dim=10000, input_length=100)(main_input)
  
  lstm1 = LSTM(50,input_shape=(1,xtrain.shape[1]),return_sequences=True)
  lstmPm25 = lstm1(inputs)
  lstm2 = LSTM(50,return_sequences=True)(lstmPm25)
  lstmPm25 = Dense(1,activation='sigmoid')(lstm2)
  lstmModel1 = Model(inputs, lstmPm25)
  
  aux = Input((1,auxdata.shape[1]))
  #lstm2 = LSTM(50,input_shape=(1,auxdata.shape[1]), return_sequences=True)
  #lstmMeteo = lstm2(aux)
  #lstmMeteo = Dense(2000)(aux)
  #lstmMeteo = Dense(2000)(lstmMeteo)
  lstmMeteo = Dense(39,activation='sigmoid')(aux)
  lstmModel2 = Model(aux, lstmMeteo)
  
  combined = concatenate([lstmModel1.output, lstmModel2.output],axis=2)
  fc1 = Dense(50)(combined)
  fc2 = Dense(1, activation="sigmoid")(fc1)
  fcModel = Model(inputs=[lstmModel1.input, lstmModel2.input],outputs=fc2)
  return fcModel

def lstm(xtrain,ytrain,auxdata):
  inputs = Input((1,xtrain.shape[1]))
  
  #~inputs = Embedding(output_dim=xtrain.shape[0], input_dim=10000, input_length=100)(main_input)
  
  lstm1 = LSTM(600,input_shape=(1,xtrain.shape[1]),return_sequences=True)
  lstmPm25 = lstm1(inputs)
  lstm2 = LSTM(600,return_sequences=True)(lstmPm25)
  lstmPm25 = Dense(1,activation='sigmoid')(lstm2)
  lstmModel1 = Model(inputs, lstmPm25)

  return lstmModel1

def lstmed(xtrain,ytrain,auxdata):
  encoder_inputs = Input(shape=(1,xtrain.shape[1]))
  encoder = LSTM(36, return_state=True)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  encoder_outputs = Dense(1, activation='sigmoid')(encoder_outputs)
  #encoder_outputs = tf.keras.backend.mean(encoder_outputs,axis=1)
  encoder_states = [state_h, state_c]
  
  decoder_inputs = Dense(36)(encoder_outputs)
  decoder_inputs = Reshape((1,36))(decoder_inputs)
  
  decoder_lstm = LSTM(36, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  decoder_dense = Dense(1, activation='sigmoid')
  decoder_outputs = decoder_dense(decoder_outputs)
  
  model = Model(encoder_inputs, decoder_outputs)
  return model 

def randomForest(xtrain,ytrain,xtest):
  regressor = RandomForestRegressor(n_estimators=100, random_state=0)  
  regressor.fit(xtrain, ytrain)  
  y_pred = regressor.predict(xtest)  
  return y_pred

In [34]:
kFolds = KFold(n_splits=5)
countCross = 1 
for train, test in kFolds.split(fulldata):
    xTrain = fulldata[train,0:8]
    xTest= fulldata[test,0:8]
    auxTrain = fulldata[train,8:47]
    auxTest = fulldata[test,8:47] 
    yTrain = fulldata[train,47:48]
    yTest = fulldata[test,47:48]
    print(auxTrain.shape)
    print("------------- CROSS VALIDATION ",countCross,"---------")
    print("**********LTSME***********")
    model = lstme(xTrain,yTrain,auxTrain)
    print(model.summary())
    model.compile(loss='mae', optimizer='adam') 
    xTrain = np.reshape(xTrain,(xTrain.shape[0],1,xTrain.shape[1]))
    auxTrain = np.reshape(auxTrain,(auxTrain.shape[0],1,auxTrain.shape[1]))
    yTrain = np.reshape(yTrain,(yTrain.shape[0],1,yTrain.shape[1]))
    
    model.fit([xTrain,auxTrain], yTrain, epochs=100, batch_size=1000)
    
    xTest = np.reshape(xTest,(xTest.shape[0],1,xTest.shape[1]))
    auxTest = np.reshape(auxTest,(auxTest.shape[0],1,auxTest.shape[1]))
    
    yhat = model.predict([xTest,auxTest])
    
    yhat = np.reshape(yhat,(xTest.shape[0],xTest.shape[1]))
    predictTest = scaler.inverse_transform(yhat)
    yTestI = scaler.inverse_transform(yTest)
    rmseTest = sqrt(mean_squared_error(yTestI, predictTest))
    print("RMSE ", rmseTest)
    countCross+=1

(40000, 39)
------------- CROSS VALIDATION  1 ---------
**********LTSME***********
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 1, 8)         0                                            
__________________________________________________________________________________________________
lstm_15 (LSTM)                  (None, 1, 50)        11800       input_15[0][0]                   
__________________________________________________________________________________________________
lstm_16 (LSTM)                  (None, 1, 50)        20200       lstm_15[0][0]                    
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 1, 39)        0                                            
__________________________

KeyboardInterrupt: ignored

In [0]:
kFolds = KFold(n_splits=5)
countCross = 1 
for train, test in kFolds.split(fulldata):
    xTrain = fulldata[train,0:8]
    xTest= fulldata[test,0:8]
    auxTrain = fulldata[train,8:47]
    auxTest = fulldata[test,8:47] 
    yTrain = fulldata[train,47:48]
    yTest = fulldata[test,47:48]
    print(auxTrain.shape)
    print("------------- CROSS VALIDATION ",countCross,"---------")
    print("**********LTSM NN***********")
    model = lstm(xTrain,yTrain,auxTrain)
    print(model.summary())
    model.compile(loss='mae', optimizer='adam') 
    xTrain = np.reshape(xTrain,(xTrain.shape[0],1,xTrain.shape[1]))
    auxTrain = np.reshape(auxTrain,(auxTrain.shape[0],1,auxTrain.shape[1]))
    yTrain = np.reshape(yTrain,(yTrain.shape[0],1,yTrain.shape[1]))
    
    model.fit(xTrain, yTrain, epochs=10, batch_size=1000)
    
    xTest = np.reshape(xTest,(xTest.shape[0],1,xTest.shape[1]))
    auxTest = np.reshape(auxTest,(auxTest.shape[0],1,auxTest.shape[1]))

    yhat = model.predict(xTest)
    
    yhat = np.reshape(yhat,(xTest.shape[0],xTest.shape[1]))
    predictTest = scaler.inverse_transform(yhat)
    yTestI = scaler.inverse_transform(yTest)
    rmseTest = sqrt(mean_squared_error(yTestI, predictTest))
    print("RMSE ", rmseTest)
    countCross+=1

In [0]:
kFolds = KFold(n_splits=5)
countCross = 1 
for train, test in kFolds.split(fulldata):
    xTrain = fulldata[train,0:8]
    xTest= fulldata[test,0:8]
    auxTrain = fulldata[train,8:47]
    auxTest = fulldata[test,8:47] 
    yTrain = fulldata[train,47:48]
    yTest = fulldata[test,47:48]
    print(auxTrain.shape)
    print("------------- CROSS VALIDATION ",countCross,"---------")
    print("**********LTSMED***********")
    model = lstmed(xTrain,yTrain,auxTrain)
    print(model.summary())
    model.compile(loss='mae', optimizer='adam') 
    xTrain = np.reshape(xTrain,(xTrain.shape[0],1,xTrain.shape[1]))
    auxTrain = np.reshape(auxTrain,(auxTrain.shape[0],1,auxTrain.shape[1]))
    yTrain = np.reshape(yTrain,(yTrain.shape[0],1,yTrain.shape[1]))
    
    model.fit(xTrain, yTrain, epochs=100, batch_size=10)
    
    xTest = np.reshape(xTest,(xTest.shape[0],1,xTest.shape[1]))
    auxTest = np.reshape(auxTest,(auxTest.shape[0],1,auxTest.shape[1]))
    
    yhat = model.predict([xTest])
    
    yhat = np.reshape(yhat,(xTest.shape[0],xTest.shape[1]))
    predictTest = scaler.inverse_transform(yhat)
    yTestI = scaler.inverse_transform(yTest)
    rmseTest = sqrt(mean_squared_error(yTestI, predictTest))
    print("RMSE ", rmseTest)
    countCross+=1

In [33]:
kFolds = KFold(n_splits=5)
countCross = 1 
for train, test in kFolds.split(fulldata):
    xTrain = fulldata[train,0:8]
    xTest= fulldata[test,0:8]
    auxTrain = fulldata[train,8:47]
    auxTest = fulldata[test,8:47] 
    yTrain = fulldata[train,47:48]
    yTest = fulldata[test,47:48]
    prediction = randomForest(xTrain,yTrain,xTest)
    print("------------- CROSS VALIDATION ",countCross,"---------")
    print("**********Random Forest***********")
    prediction = np.reshape(prediction,(-1, 1))
    predictTest = scaler.inverse_transform(prediction)
    yTestI = scaler.inverse_transform(yTest)
    rmseTest = sqrt(mean_squared_error(yTestI, predictTest))
    print("RMSE ", rmseTest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


------------- CROSS VALIDATION  1 ---------
**********Random Forest***********
RMSE  26.863240132366634


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


------------- CROSS VALIDATION  1 ---------
**********Random Forest***********
RMSE  25.313278031021664


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


------------- CROSS VALIDATION  1 ---------
**********Random Forest***********
RMSE  27.634283180376137


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


------------- CROSS VALIDATION  1 ---------
**********Random Forest***********
RMSE  21.573255586555174


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


------------- CROSS VALIDATION  1 ---------
**********Random Forest***********
RMSE  20.481059177876855
